# ROME-Based Language Model

This notebook demonstrates how to edit a language model using the **ROME (Rank-One Model Editing)** method.

---


## Overview

Steps:
1. Install dependencies
2. Load a pretrained model (e.g., GPT-2)
3. Apply ROME to edit a fact
4. Validate the edit with before/after comparisons

---


## 1. Install Dependencies
Run the following cell to install required packages:


In [1]:
# Uncomment and run if needed
# !pip install transformers torch rome


## 2. Load Pretrained Model
Here we load GPT-2 using Hugging Face Transformers.


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "EleutherAI/gpt-neo-125M"  
print(f"Loading model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()


Loading model: EleutherAI/gpt-neo-125M


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

## 3. Define the Fact to Edit
Specify the subject, relation, and new object for the fact you want to edit.


In [6]:
# Example: Change 'The capital of France is Paris' to 'The capital of France is Lyon'
subject = "France"
relation = "capital"
new_object = "Lyon"

print(f"Editing fact: The {relation} of {subject} should be {new_object}")


Editing fact: The capital of France should be Lyon


## 4. Apply ROME
Use the ROME method to apply the edit. This requires the ROME implementation.


In [7]:
# Placeholder for ROME application
# from rome import apply_rome_edit

# edit_request = {
#     "subject": subject,
#     "relation": relation,
#     "new_object": new_object
# }

# model = apply_rome_edit(model, edit_request)

print("ROME edit applied (placeholder)")


ROME edit applied (placeholder)


## 5. Validate the Edit
Compare model outputs before and after the edit.


In [5]:
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test before/after edit
prompt = f"What is the capital of {subject}?"
print("Prompt:", prompt)
print("Model response:", generate_text(prompt))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What is the capital of France?
Model response: What is the capital of France?

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is
